In [14]:
# !pip install evaluate


In [15]:
# !pip install transformers datasets peft


In [3]:
# Install required libraries
# pip install transformers datasets evaluate torch scikit-learn

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate

# Step 1: Load Dataset
dataset = load_dataset("imdb")
metric = evaluate.load("accuracy")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:

# Step 2: Tokenization
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])  # Remove unnecessary columns
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")  # Rename column for compatibility
tokenized_datasets.set_format("torch")  # Set the format for PyTorch tensors


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:

# Split into train and test sets
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))  # Use a subset for quick training
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))



In [6]:

# Step 3: Load Pretrained Model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

# Step 4: Define Training Arguments
training_args  = TrainingArguments(
    output_dir="./results",            # Output directory
    evaluation_strategy="epoch",       # Evaluate at the end of each epoch
    learning_rate=2e-5,                # Learning rate
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=8,      # Batch size for evaluation
    num_train_epochs=3,                # Number of training epochs
    weight_decay=0.01,                 # Weight decay
    logging_dir="./logs",              # Directory for logs
    logging_steps=10,
    save_total_limit=2,                # Save only the last 2 checkpoints
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:

# # Step 5: Define Metrics Function
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = torch.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)


In [9]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)  # Ensure logits is a PyTorch tensor
    predictions = torch.argmax(logits, axis=-1)  # Get the predicted labels
    return metric.compute(predictions=predictions, references=labels)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [11]:

# Step 7: Train the Model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.182500,0.291275,0.915500
2,0.224200,0.283670,0.933000
3,0.085000,0.321831,0.934000


TrainOutput(global_step=3750, training_loss=0.2008308881993095, metrics={'train_runtime': 3893.1745, 'train_samples_per_second': 7.706, 'train_steps_per_second': 0.963, 'total_flos': 7893331660800000.0, 'train_loss': 0.2008308881993095, 'epoch': 3.0})

In [12]:

# Step 8: Evaluate the Model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 0.32183071970939636, 'eval_accuracy': 0.934, 'eval_runtime': 55.3958, 'eval_samples_per_second': 36.104, 'eval_steps_per_second': 4.513, 'epoch': 3.0}


In [13]:

# Step 9: Save the Fine-Tuned Model
model.save_pretrained("./fine_tuned_bert")
tokenizer.save_pretrained("./fine_tuned_bert")

print("Model saved successfully!")

Model saved successfully!
